-------------------------------------------------------------------------------------------------------------------
Implementation of : "Online Label Recovery for Deep Learning-based communication through Error Correcting codes"

Author : Eric Soubigou

Date : Spring 2019

-------------------------------------------------------------------------------------------------------------------

Description :  Creation of a DFE like with Deep Learning technologies

In [1]:
# Install libraries :
!pip3 install --user git+git://github.com/veeresht/CommPy.git@master
!pip3 install --user torch
!pip3 install --user matplotlib
!pip3 install --user scipy
print("================== DONE ! ==================")

  Cloning git://github.com/veeresht/CommPy.git (to revision master) to /tmp/pip-req-build-gt688dwr
  Stored in directory: /tmp/pip-ephem-wheel-cache-923ei1hu/wheels/d1/6a/31/8ddc70e8eb8a1c3ad344032ed43b4ebfccc41007e8850226d0
Successfully built scikit-commpy
================== DONE ! ==================


In [15]:
## Imports
from __future__ import print_function
import matplotlib.pyplot as plt
import pickle # For saving file
import copy as cpy

# Scipy
import scipy as sp
from scipy import signal

# Numpy
import numpy as np
np.set_printoptions(precision=2)

# Random
import random

# Compy
from commpy.filters import *
import commpy as cp
## Simulation import
from commpy.channelcoding.convcode import Trellis, conv_encode, viterbi_decode
from commpy.modulation import *

# For DL libraries 
import torch
import torch.nn as nn
import torch.nn.functional as F


if torch.cuda.is_available():
    device = torch.device('cuda')
    FloatTensor = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')
    FloatTensor = torch.FloatTensor


In [3]:
""" Return if the number is a power of two. 
:num: An integer, number to test.
"""
def is_power_of_2(num):
    return num != 0 and ((num & (num - 1)) == 0)

""" Plot the power density spectrum of a given signal
:signal: A 1D-float-array, with the signal samples
:time_step: An integer, time step of the signal sampling 
"""
def plot_spectrum(signal, time_step):
    # Go in the frequency domain
    spectrum = np.abs(np.fft.fftshift(np.fft.fft(signal))) ** 2
    #   f, welch_estimate = sp.signal.welch(signal)
    freq = np.fft.fftshift(np.fft.fftfreq(signal.size, d=time_step))
    plt.plot(freq, spectrum, "r")
    #   plt.plot(f, welch_estimate, 'b')
    plt.yscale("log")
    plt.title("OFDM Spectrogram")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()

In [4]:
"""
  Emitter class
"""


class Emiter:
    """ Class of the emiter.
  
  :cp_len: An integer, length of the cyclic-prefix of the OFDM.
  :nb_carriers: An integer, number of sub_carrier used to transmit data.
  :modulation_order: An integer, the modulation order.
  :nb_off_carriers: An integer, number of off carrier in OFDM scheme
  :trellis: A compy.channelcoding.Trellis, to encode the data
  :pilot_frequency: A integer, number of OFDM symbols/frame before transmitting a pilot
  
  Ex : | CP | CP | OFF | OFF | ON | ON | ... | ON |
  """

    def __init__(
        self,
        cp_len,
        nb_carriers,
        modulation_order,
        trellis,
        nb_off_carriers=0,
        pilot_frequency=8,
    ):
        self.cp_len = cp_len
        self.nb_carriers = nb_carriers
        self.nb_off_carriers = nb_off_carriers
        # Number of carriers that are used knowing the number of off-carrier
        self.nb_on_carriers = self.nb_carriers - self.nb_off_carriers

        # For the pilot management
        self.pilot_frequency = pilot_frequency

        if is_power_of_2(modulation_order):
            self.modulator = cp.modulation.PSKModem(modulation_order)
        else:
            print("Wrong modulation order : modulation order = ", modulation_order)

        # Generate OFDM pilot symbol
        self._pilot_symbols = np.expand_dims(
            self.modulator.modulate(
                np.zeros(
                    self.nb_on_carriers * int(np.log2(modulation_order)), dtype=int
                )
            ),
            axis=1,
        ).T

        # Test if the trellis is well-defined
        if trellis is not None:
            self.enc_trellis = trellis
        else:
            print("trellis is not defined")

    def get_trellis(self):
        """ Return the copy of trellis of the emiter
        """
        return cpy.deepcopy(self.enc_trellis)

    def modulate_frame(self, frame):
        """ Modulate and Map the frame. In other words, will perform the 
        Modulation and the Mapping and then perform the OFDM transmormation before 
        sending the data.
        :frame: The frame that has to be modulated.
        """
        # Mapping of the data
        mod_frame = self.modulator.modulate(frame)

        # Test if the division is equal to an integer or not
        if len(mod_frame) % self.nb_on_carriers != 0:
            nb_ofdm_group = (len(mod_frame) // self.nb_on_carriers) + 1
            # Add padding to the frame in order to get a interger number of PHY
            # frames used.

            padding = np.zeros(
                (self.nb_on_carriers - len(mod_frame) % self.nb_on_carriers)
                * self.modulator.num_bits_symbol,
                dtype=int,
            )
            # Add padding to the modulated frame
            mod_frame = np.concatenate(
                (mod_frame, self.modulator.modulate(padding)), axis=0
            )
        else:
            # No padding to perform
            nb_ofdm_group = len(mod_frame) // self.nb_on_carriers

        # Then reshape the frame to perform the modulation
        carriers = np.reshape(mod_frame, (nb_ofdm_group, self.nb_on_carriers))

        # Insert the pilot symbols at the given frequency along rows
        carriers = np.insert(
            carriers,
            np.arange(0, carriers.shape[0], self.pilot_frequency),
            self._pilot_symbols,
            axis=0,
        )

        # Test if there are some off carriers
        if self.nb_off_carriers > 0:
            # Add the off_carriers
            carriers = np.concatenate(
                (np.zeros((nb_ofdm_group, self.nb_off_carriers)), carriers), axis=1
            )
            
        # Then use the matrix to transform them into OFDM symbols
        ofdm_signal = sp.fftpack.ifft(carriers, axis=1)
        #  Add the cyclic prefix
        ofdm_signal_cp = np.concatenate(
            (ofdm_signal[:, self.nb_carriers - self.cp_len :], ofdm_signal), axis=1
        )
        # Return the global modulated frame
        return np.ravel(ofdm_signal_cp)

    def encode(self, frame):
        """ Encode the bit frame according to the defined trellis of the emiter
        :frame: The frame that has to be encoded.
        """
        # Channel coding of the frame
        return cp.channelcoding.conv_encode(frame, self.enc_trellis)

In [5]:
"""
  AWGN Channel class
"""


class AWGN_Channel:

    """ Class of the AWGN channel.
    :mean: A float, mean of the gaussian noise.
    :var: A float, variance of the gaussian noise.
    :non_lin_coeff: A float, value of the non-linearity coefficient of the channel.
    :iq_imbalance: A float, value of the iq_imbalance.
    :channel_taps: A 1-D float-array, containing the value of channel's taps.
    :up_factor: A positive integer, value of the upsampling factor
    """

    def __init__(
        self,
        mean,
        var,
        non_lin_coeff=0,
        iq_imbalance=None,
        channel_taps=np.ones(1),
        up_factor=1,
    ):
        self.mean = mean
        self.var = var
        self.gamma = non_lin_coeff
        self.beta = iq_imbalance
        self.channel_taps = np.divide(channel_taps, np.linalg.norm(channel_taps))
        self.up_factor = up_factor

    def get_trough(self, mod_frame):
        """ Return the value of the frame after get through the channel
        :mod_frame: An array, input of of the channel. Is the modulated frame from an 
        emiter.
        """
        # Add the IQ imbalance at the emiter side
        mod_frame = self.add_iq_imbalance(mod_frame)
        # Add the non linearity effect at the emiter side
        mod_frame -= self.gamma * (np.abs(mod_frame)) ** 2 * mod_frame
        # Go through the multipath channel if the response length is greater than one tap
        if len(self.channel_taps) > 1:
            # Perform the channel filtering
#             print("mod_frame before up-sampling", mod_frame.shape)
#             print("Mod frame output: ", mod_frame[6:20])
            #mod_frame = sp.signal.resample(mod_frame, len(mod_frame)*self.up_factor)
#             print("mod_frame after up-sampling", mod_frame.shape)
#             print("Mod frame output: ", mod_frame[6:20])
            # Filter
#             print("Channel taps", self.channel_taps)
            mod_frame = sp.signal.lfilter(b=self.channel_taps, a=[1], x=mod_frame)
#             print("mod_frame after filtering", mod_frame.shape)
#             print("Mod frame output: ", mod_frame[6:20])
            # Down sample the signal
            #mod_frame = sp.signal.decimate(mod_frame, q=self.up_factor)
#             print("mod_frame after decimation", mod_frame.shape)
#             print("Mod frame output: ", mod_frame[6:20])
#             mod_frame = sp.signal.upfirdn(
#                 self.channel_taps, mod_frame, up=self.up_factor, down=self.up_factor
#             )
            # Then shrink the reponse
            #mod_frame = mod_frame[len(self.channel_taps)-1:]
        # Add Gaussian noise
        output = mod_frame + np.random.normal(self.mean, self.var, mod_frame.shape)
        # Add the second IQ imbalance at the receiver side
        output = self.add_iq_imbalance(output)
        # Add the non linearity effect at the receiver side
        output -= self.gamma * np.abs(output) ** 2 * output
        return output

    def add_iq_imbalance(self, x):
        """ Add IQ imbalance to a given array
        :x: An array, input which will be imbalanced accodring to attributes of the 
        AWGN channel class
        """
        if self.beta is not None:
            return self.beta * np.real(x) + 1j * np.imag(x)
        else:
            return x

    def set_var(self, snr_db, modulation_order):
        """ Set the variance of the gaussian noise of the channel
        :snr_db: A float, Value of the Signal to Noise ratio in dB
        :modulation_order: An integer, is the moudlation order of the constellation used
        """
        snr = 10 ** (snr_db / 10)
        # To define a bit better than it is right now
        var_signal = 1
        shaped_signal = 1
        self.var = (
            np.linalg.norm(shaped_signal)
            * var_signal
            * np.linalg.norm(self.channel_taps)
        ) / (2 * np.log2(modulation_order) * snr)

In [30]:
class PreEqualizer(nn.Module):
    """ Neural net of the pre-equlizer
    :fc_n: fully connected layers
    """

    def __init__(self, symb_nb):
        super(Net, self).__init__()
        # Set the loss as the MSE loss function. (Not the Cross entropy)
        self.loss_function = nn.MSELoss
        # Number of symbols accepted in the entry of the neural network
        self.symb_nb = symb_nb
        #  Fully connected layers
        self.fc1 = nn.Linear(2 * self.symb_nb, 256)
        self.fc2 = nn.Linear(2 * self.symb_nb, 256)
        self.fc3 = nn.Linear(2 * self.symb_nb, 256)
        # Init trainable scalar
        self.alpha = torch.randn(1, 1)

    def forward(self, symbols):
        """ Perform the feedforwar process 
        :symbols: A 1D float array, containing the symbols to pre-equalize
        """
        # First, adapat the input to the Neural network this means that we
        # have to handle the case where there is the CP
        pre_equ_symbols = np.array(symbols.shape, dtype=complex)
        # Loop on the different chunk of the signal
        for i in range(len(symbols) / self.symb_nb):
            # Convert the complex array to a 2D real array
            formated_symb = from_complex_to_real(
                symbols[i * self.symb_nb : (i + 1) * self.symb_nb]
            )
            # Then feed the neural network with the adapted symbol
            out_1 = F.relu(self.fc1(formated_symb))
            out_2 = F.relu(self.fc2(out_1))
            out_3 = F.linear(self.fc3(out_2))
            # Convert the output to a complex vector.
            pre_equ_symbols[
                i * self.symb_nb : (i + 1) * self.symb_nb
            ] = from_real_to_complex(wght_out)
        # Lastly we sum the complex input with the ponderated ouptut of the network
        return symbols + self.alpha * pre_equ_symbols

    def backpropagation(self, output_symb, targeted_symb):
        """ Use to perform the back propagation update
        :output_symb:
        :targeted_symb:
        """
        loss = self.loss_function(output_symb, targeted_symb)
        loss.backward()

    def feedback_update(self, x_hat):
        """ Perform a SGD to update the weights given the results of 
        the viterbi decoder
        :x_hat: TODO !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        """
        x_hat = 1

    # Static Methods

    @staticmethod
    def train(pre_equalizer, received_symb, targeted_symb, nb_epochs, sgd_step=0.01):
        """ To train the NN pre-equalizer.
        :pre_equalizer: A Pre_Equalizer, the one which will be trained
        :received_symb: A 1D array, received symbols from the OFDM
        :targeted_symb: A 1D array of the targeted symbols before the 
            demaping process
        TODO !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        """
        optimizer = optim.SGD(pre_equalizer.parameters(), lr=sgd_step)
        # Loop on the number of epcohs
        for epoch in range(nb_epochs):
            for i, mini_batch in enumerate(data, 0):
                # Zero the gradient buffers
                optimizer.zero_grad()
                # Perform the forward operation
                pre_eq_symbols = pre_equalizer.forward(received_symb)
                # Launch the backward function
                pre_equalizer.backpropagation(pre_eq_symbols, targeted_symb)
                # Perform update of the gradient
                optimizer.step()

def from_real_to_complex(symbols):
    """ Convert a vector of real 2N symbol into a N complex vetor
    with alternate real/imag part of the symbols
    :symbols: A 2D-real array, containing the value of symbols.
    """
    # Init of the vector
    cmplx = np.array((1, np.round(len(symbols) / 2)), dtype=complex)
    cmplx = symbols[0:2:] + j * symbols[1:2:]
    return cmplx

def from_complex_to_real(symbols):
    """ Convert a vector of complex N symbol into a 2N real vetor
    with alternate real/imag part of the symbols
    :symbols: A 1D-complex array, containing the value of symbols.
    """
    return np.ravel(np.concatenate((np.real(symbols), np.imag(symbols)), axis=1))

In [7]:
"""
    Equalizer class
"""


class Equalizer:
    """ Class of Equalizers
    :pilot_symbols: An 1D-float array, beeing the pilots symbol used to estimate 
    """

    def __init__(self, pilot_symbols):
        self.pilot_symbols = pilot_symbols
        self.estimation = None
        self._name = "None"

    def equalize(self, symbols_to_equalize):
        """ Equalizes the received frame
        :symbols_to_equalize: A 2D-float-array, with the symbol to equalize
        """
        return np.divide(symbols_to_equalize, self.estimation)

    def get_name(self):
        return self._name


class Zero_Forcing(Equalizer):
    """ Zero Forcing equalizers class
    """

    def __init__(self, pilot_symbols):
        super().__init__(pilot_symbols)
        self._name = "Zero-Forcing"

    def estimate(self, received_pilot_symbols):
        """
        :received_pilot_symbols: A 1D-complex-array, containing the signal samples of the 
            received pilot symbols.
        """
        self.estimation = np.divide(
            np.multiply(received_pilot_symbols, np.conjugate(self.pilot_symbols)),
            np.power(np.abs(self.pilot_symbols), 2),
        )


class MMSE(Equalizer):
    """ MMSE equalizers class
    :noise_var_est: A float, value of the gaussian noise variance.
    """

    def __init__(self, pilot_symbols):
        super().__init__(pilot_symbols)
        self._name = "MMSE"
        self._noise_var_est = 0

    def estimate(self, received_pilot_symbols):
        """ Estimate the channel taps following the MMSE principles
        :received_pilot_symbols: A 1D-complex-array, containing the signal samples of the 
            received pilot symbols.
        """
        self.estimation = np.divide(
            np.multiply(received_pilot_symbols, np.conjugate(self.pilot_symbols)),
            np.linalg.norm(self.pilot_symbols) ** 2 + self._noise_var_est,
        )

    def set_noise_var(self, noise_var_est):
        """ Set the value of the gaussian noise variance
        :noise_var_est: A float, value of the noise variance estimated.
        """
        self._noise_var_est = noise_var_est


class NN_Equalizer(Equalizer):
    def __init__(self, pilot_symbols):
        super().__init__(pilot_symbols)
        self._name = "Zero-Forcing"

        # Instantiate the neural network


def switch_init_equalizer(equalizer_name, pilot_symbols):
    """ Instantiate the wanted equalizer given the name of the equalizer
    """
    equalizers = {
        "ZF": Zero_Forcing(pilot_symbols=pilot_symbols),
        "MMSE": MMSE(pilot_symbols=pilot_symbols),
    }
    return equalizers.get(equalizer_name, None)

In [8]:
"""
    Receiver class
"""


class Receiver:

    """ Class of the Receivers
    
    :cp_len: An integer, length of the cyclic-prefix of the OFDM.
    :nb_carriers: An integer, number of sub_carrier used to transmit data.
    :modulation_order: An integer, the modulation order.
    :nb_off_carriers: An integer, number of off carrier in OFDM scheme
    """

    def __init__(
        self,
        cp_len,
        nb_carriers,
        modulation_order,
        trellis,
        nb_off_carriers=0,
        pilot_frequency=8,
        equalizer_type=None,
    ):
        self.cp_len = cp_len
        self.nb_carriers = nb_carriers
        self.nb_off_carriers = nb_off_carriers
        # Number of carriers that are used knowing the number of off-carrier
        self.nb_on_carriers = self.nb_carriers - self.nb_off_carriers

        # For the pilot management
        self.pilot_frequency = pilot_frequency

        if is_power_of_2(modulation_order):
            self.demodulator = cp.modulation.PSKModem(modulation_order)
        else:
            print("Wrong modulation order : modulation order = ", modulation_order)

        # Generate OFDM pilot symbol to use it for future equalization processes
        pilot_symbols = np.expand_dims(
            self.demodulator.modulate(
                np.zeros(
                    self.nb_on_carriers * int(np.log2(modulation_order)), dtype=int
                )
            ),
            axis=1,
        ).T

        # Test if the trellis is well-defined
        if trellis is not None:
            self.dec_trellis = trellis
        else:
            print("trellis is not defined")

        # Instanciate the Equalizer
        if equalizer_type is not None:
            # Test which type of equalizer
            self.equalizer = switch_init_equalizer(equalizer_type, pilot_symbols)
        else:
            self.equalizer = None

    def demodulate_frame(self, frame, demod_type="hard"):
        """ Demodulate a OFDM received frame
        :frame: A 1D array, received frame to demodulate (following OFDM scheme).
        :demod_type: 'hard'/'soft', type of demodulation wanted.
        """
        # We reshape the frame at the reception
        nb_ofdm_group = len(frame) // (self.cp_len + self.nb_carriers)
        received_frame = np.reshape(
            frame, (nb_ofdm_group, (self.cp_len + self.nb_carriers))
        )
        # We delete the cyclic prefix from each frame
        received_frame = received_frame[:, self.cp_len :]
        # We perform the fft to go from OFDM modulation to standard maping
        time_frame = sp.fftpack.fft(received_frame, axis=1)
        # Then we delete the part of off-carriers
        time_frame = time_frame[:, self.nb_off_carriers :]

        # Every two OFDM symbol, we have to extract the pilots symbols of every two frame
        # and used it for eqaulization
        idx_to_extract = np.arange(0, time_frame.shape[0], self.pilot_frequency + 1)
        received_pilot_symbols = time_frame[idx_to_extract, :]
        #  And delete the pilot symbols
        time_frame = np.delete(time_frame, idx_to_extract, 0)

        # Use of an equalizer ?
        if self.equalizer is not None:
            # Create the new equalized frame which is
            eq_time_frame = np.empty((0, time_frame.shape[1]), dtype=complex)
            # We iterate over the different pilot symbols
            for pilot_idx in range(0, received_pilot_symbols.shape[0]):
                # Set the new estimation
                self.equalizer.estimate(received_pilot_symbols[pilot_idx, :])
                idx = np.arange(
                    self.pilot_frequency * pilot_idx,
                    np.minimum(
                        self.pilot_frequency * (pilot_idx + 1), time_frame.shape[0]
                    ),
                )
                eq_sub_frame = self.equalizer.equalize(time_frame[idx, :])
                # Append the equalized results the new equalized frame
                eq_time_frame = np.append(eq_time_frame, eq_sub_frame, axis=0)
        else:
            # No equalization performed
            eq_time_frame = time_frame

        # Reshape the time frame in a 1D-array
        eq_time_frame = np.ravel(eq_time_frame)

        # Then we perform the demodulation
        return self.demodulator.demodulate(eq_time_frame, demod_type)

    def decode(self, enc_frame):
        """ Decoding an encoded frame according to the trellis of the receiver.
        :enc_frame: A 1D float array, encoded frame to decode.
        """
        # Decode the received frame according to the trellis
        return cp.channelcoding.viterbi_decode(
            enc_frame, self.dec_trellis, decoding_type="hard"  # , tb_length=15
        )

In [ ]:
# """
#   Unitary test of Emiter/Receiver class
# """
# # Use of seed
# random.seed(2019)
# np.random.seed(2019)

# # Channel code parameters + trellis generation
# mem_size = np.array([2])
# g_matrix = np.array([[0o5, 0o7]])
# trellis = Trellis(mem_size, g_matrix)

# #  Modulation parameters
# modulation_order = 8
# frame_length = 100

# # OFDM parameters (According to the paper "Online Label Recovery for Deep
# # Learning-based communication through Error Correcting codes")
# nb_carriers = 64
# cp_length = 8
# off_carrier = 0
# equalizer = "ZF"  # "ZF" #

# # Creation of the emiter
# emiter = Emiter(
#     cp_length, nb_carriers, modulation_order, trellis, nb_off_carriers=off_carrier
# )

# # Basic unitary test
# print("Modulation exponent: ", emiter.modulator.num_bits_symbol)

# # Vizualization of trellis
# (emiter.get_trellis()).visualize()

# ### Test the frame modulation
# # Generation of a frame
# frame = np.random.randint(0, high=2, size=(frame_length))
# print("The frame length is : ", frame.shape)

# # Channel Coding by the emiter
# enc_frame = emiter.encode(frame)
# print("The encoded frame length is : ", enc_frame.shape)

# # Modulation operation by the emiter
# mod_frame = emiter.modulate_frame(enc_frame)
# print("The modulated frame shape is : ", mod_frame.shape)

# # Verification of the spectrum :
# plot_spectrum(mod_frame, 1)

# # Get through channel
# mean = 0
# var = 0.3
# awgn_channel = AWGN_Channel(
#     mean,
#     var,
#     non_lin_coeff=0,
#     iq_imbalance=None,
#     channel_taps=np.array([1, 2, 3, 2, 1]),  # np.array([1, 2, 3, 2, 1]),
#     up_factor=1,  # nb_carriers + cp_length, TOMODIFY
# )

# received_frame = awgn_channel.get_trough(mod_frame)

# # Creation of the receiver
# receiver = Receiver(
#     cp_length,
#     nb_carriers,
#     modulation_order,
#     trellis,
#     nb_off_carriers=off_carrier,
#     equalizer_type=equalizer,
# )

# # Demodulation of the received frame
# demod_frame = receiver.demodulate_frame(received_frame, demod_type="hard")
# print("The demodulated frame length is : ", demod_frame.shape)

# # Shrink for convinience
# demod_frame = demod_frame[: len(enc_frame)]
# print("The demodulated frame length is : ", demod_frame.shape)

# # Comparing the error
# nb_err_demod = np.sum(demod_frame != enc_frame)
# print("The number of error after demodulation is :", nb_err_demod)

# # Decoding frame
# dec_frame = receiver.decode(demod_frame)
# print("The decoded frame length is : ", dec_frame.shape)

# # Shrink the last part of the decoded frame before comparing the results
# dec_frame = dec_frame[: len(frame)]
# print("The shrink decoded frame length is : ", dec_frame.shape)

# # Comparing the error
# nb_err = np.sum(dec_frame != frame)
# print("The number of error is :", nb_err)

In [9]:
class PHY_Layer:
    """ PHY layer class
    :emiter: An Emiter, emiter of the Phy layer
    :receiver: An Receiver, receiver of the Phy layer
    :channel: A Channel, 
    """

    def __init__(self, emiter, receiver, channel):
        self.emiter = emiter
        self.receiver = receiver
        self.channel = channel

    def process_frame(self, frame):
        # Encode the frame
        enc_frame = self.emiter.encode(frame)
        # Modulate the frame
        mod_frame = self.emiter.modulate_frame(enc_frame)
        # Go through the channel
        channel_frame = self.channel.get_trough(mod_frame)
        # Demodulation of the received frame
        demod_frame = self.receiver.demodulate_frame(channel_frame, demod_type="hard")
        # Shrink for convinience
        demod_frame = demod_frame[: len(enc_frame)]
        # Decoding frame
        dec_frame = self.receiver.decode(demod_frame)
        # Shrink the last part of the decoded frame before comparing the results
        return dec_frame[: len(frame)]

In [ ]:
# ### Monte-carlo simulations

# # Use of seed
# random.seed(2019)
# np.random.seed(2019)

# # Monte-Carlo parameters
# min_error_frame = 100
# targeted_fer = 10e-3
# max_test = min_error_frame / targeted_fer

# # SNR / BER parameters
# step_db = 2
# min_eb_n0 = 0
# max_eb_n0 = 36
# eb_n0_db = np.array(range(min_eb_n0, max_eb_n0, step_db))
# ber = np.zeros(len(eb_n0_db))
# fer = np.zeros(len(eb_n0_db))
# ser = np.zeros(len(eb_n0_db))

# # Channel parameters
# non_lin_coeff = 0
# iq_imbalance = None
# channel_taps = np.array([1, 2, 3, 2, 1])  # proakis C

# # Modulation order of the constellation
# modulation_order = 4

# # OFDM parameters (According to the paper "Online Label Recovery for
# # Deep Learning-based communication through Error Correcting codes")
# nb_carriers = 64
# cp_length = 8
# off_carrier = 0

# # Type of Equalizer at the receiver side [None, ZF, MMSE]
# equalizer = "MMSE"  # "ZF"

# # Emiter / Receiver parameter for trellis
# mem_size = np.array([2])
# g_matrix = np.array([[0o5, 0o7]])
# trellis = Trellis(mem_size, g_matrix)
# rho = 1 / 2  #  Coding rate

# # Compute the snr_db vector
# snr_db = eb_n0_db + 10 * np.log(rho * np.log2(modulation_order))

# # Frame length
# frame_length = 1000

# # Creation of the emiter
# emiter = Emiter(
#     cp_length, nb_carriers, modulation_order, trellis, nb_off_carriers=off_carrier
# )
# # Creation of the receiver
# receiver = Receiver(
#     cp_length,
#     nb_carriers,
#     modulation_order,
#     trellis,
#     nb_off_carriers=off_carrier,
#     equalizer_type=equalizer,
# )

# # Creation of the AWGN Channel
# awgn_channel = AWGN_Channel(
#     mean=0,
#     var=0,
#     non_lin_coeff=non_lin_coeff,
#     iq_imbalance=iq_imbalance,
#     channel_taps=channel_taps,
# )

# # Creation of the PHY Layer
# phy_layer = PHY_Layer(emiter, receiver, awgn_channel)

# ind_eb_n0 = 0
# while (ind_eb_n0 < len(eb_n0_db)) and (
#     not (ind_eb_n0 > 0) or (ber[ind_eb_n0 - 1] > 0 and nb_tries < max_test)
# ):
#     # Init variables
#     nb_tries = 0
#     nb_frame_error = 0
#     global_error_nb = 0
#     # Set the snr for the channel
#     phy_layer.channel.set_var(eb_n0_db[ind_eb_n0], modulation_order)
#     # For the moment, we consider that the noise variance is not estimated
#     # but is Genie aided.
#     if equalizer == "MMSE":
#         receiver.equalizer.set_noise_var(phy_layer.channel.var)
#     # Monte-Carlo method
#     while (nb_tries < max_test) and (nb_frame_error < min_error_frame):
#         # Generation of the frames
#         frame = np.random.randint(0, high=2, size=frame_length)
#         # Send the frame to the physical layer
#         recieved_frame = phy_layer.process_frame(frame)
#         # Counting errors
#         errors_num = np.sum(recieved_frame != frame)
#         # Look at the number of mistake
#         if errors_num > 0:
#             # Add the number of frame errors
#             nb_frame_error = nb_frame_error + 1
#         global_error_nb = global_error_nb + errors_num
#         # increase the number of tries
#         nb_tries = nb_tries + 1
#     # Update error vectors
#     ber[ind_eb_n0] = global_error_nb / (nb_tries * frame_length)
#     fer[ind_eb_n0] = nb_frame_error / nb_tries
#     #     ser[ind_eb_n0] =
#     print(
#         "At ",
#         np.floor(100 * ind_eb_n0 / len(eb_n0_db)),
#         " %",
#         ", BER = ",
#         ber[ind_eb_n0],
#         ", FER = ",
#         fer[ind_eb_n0],
#         " for  Eb/N0 = ",
#         eb_n0_db[ind_eb_n0],
#         " dB",
#         ", SNR = ",
#         snr_db[ind_eb_n0],
#         "dB",
#         " nb_tries = ",
#         nb_tries,
#     )
#     # Increase the snr index
#     ind_eb_n0 += 1

# ber_dict = {
#     "trellis": {"mem_size": mem_size, "g_matrix": g_matrix},
#     "phy_param": {
#         "frame_length": frame_length,
#         "modulation_order": modulation_order,
#         "rho": rho,
#         "equalizer": equalizer,
#     },
#     "channel_param": {
#         "channel_taps": channel_taps,
#         "non_lin_coeff": non_lin_coeff,
#         "iq_imbalance": iq_imbalance,
#     },
#     "monte_carlo_param": {
#         "min_error_frame": min_error_frame,
#         "targeted_fer": targeted_fer,
#         "max_test": max_test,
#     },
#     "results": {"eb_n0_db": eb_n0_db, "snr_db": snr_db, "ber": ber, "fer": fer},
# }

# # Save results in file
# filename = "./results/OFDM_eq_{}_non_lin_coeff_{}_iq_im_{}_snr_{}_to_{}_step_{}.pickle".format(
#     str(equalizer),
#     str(non_lin_coeff),
#     str(iq_imbalance),
#     str(min_eb_n0),
#     str(max_eb_n0),
#     str(step_db),
# )

# with open(filename, "wb") as handle:
#     pickle.dump(ber_dict, handle)

# # Display results
# plt.plot(eb_n0_db, ber, "b")
# plt.yscale("log")
# plt.title("BER restults")
# plt.xlabel("Eb/N0 (dB)")
# plt.ylabel("BER")
# plt.grid(True)
# plt.show()

# plt.plot(snr_db, fer, "b")
# plt.yscale("log")
# plt.title("FER restults")
# plt.xlabel("SNR (dB)")
# plt.ylabel("FER")
# plt.grid(True)
# plt.show()

In [ ]:
## Class for the data set loading.

class DatasetSample(Dataset):
    """ Class for the data set of OFDM sample.
    Is the implementation of the abstract class Dataset
    """
    
    def __init__(self, file_path, transform=None):
        """ Intitialization of the class
        """
        self.samples = pd.rea
    
    def __getitem__(self, index):
        """ Get the sample associated with the index
        """
    
    def __len__(self):
        """ Return the length of the dataset
        """
    

In [34]:
def monte_carlo_simulation(sim_param_dict):
    """ Perform the global simulation having the dictionary of parameters.
    
    :sim_param_dict: A dictionnary containing all parameters necessary for the simualtion
    """

    max_test = (
        sim_param_dict["m_c_parameters"]["min_error_frame"]
        / sim_param_dict["m_c_parameters"]["targeted_fer"]
    )

    #  Simulation parameters
    eb_n0_db = np.array(
        range(
            sim_param_dict["m_c_parameters"]["min_eb_n0"],
            sim_param_dict["m_c_parameters"]["max_eb_n0"],
            sim_param_dict["m_c_parameters"]["step_db"],
        )
    )
    ber = np.zeros(len(eb_n0_db))
    fer = np.zeros(len(eb_n0_db))
    ser = np.zeros(len(eb_n0_db))

    # Compute the snr_db vector
    snr_db = eb_n0_db + 10 * np.log(
        sim_param_dict["channel_coding"]["rho"]
        * np.log2(sim_param_dict["modulation"]["modulation_order"])
    )

    # Creation of the trellis
    trellis = Trellis(
        sim_param_dict["channel_coding"]["mem_size"],
        sim_param_dict["channel_coding"]["g_matrix"],
    )

    # Creation of the emiter
    emiter = Emiter(
        cp_len=sim_param_dict["modulation"]["cp_length"],
        nb_carriers=sim_param_dict["modulation"]["nb_carriers"],
        modulation_order=sim_param_dict["modulation"]["modulation_order"],
        trellis=trellis,
        nb_off_carriers=sim_param_dict["modulation"]["off_carrier"],
    )

    # Creation of the receiver
    receiver = Receiver(
        cp_len=sim_param_dict["modulation"]["cp_length"],
        nb_carriers=sim_param_dict["modulation"]["nb_carriers"],
        modulation_order=sim_param_dict["modulation"]["modulation_order"],
        trellis=trellis,
        nb_off_carriers=sim_param_dict["modulation"]["off_carrier"],
        equalizer_type=sim_param_dict["equalizer"],
    )

    # Creation of the AWGN Channel
    awgn_channel = AWGN_Channel(
        mean=0,
        var=0,
        non_lin_coeff=sim_param_dict["channel_parameters"]["non_lin_coeff"],
        iq_imbalance=sim_param_dict["channel_parameters"]["iq_imbalance"],
        channel_taps=sim_param_dict["channel_parameters"]["channel_taps"],
    )

    # File name creation
    filename = "./results/OFDM_eq_{}_non_lin_coeff_{}_iq_im_{}_snr_{}_to_{}_step_{}.pickle".format(
        str(sim_param_dict["equalizer"]),
        str(sim_param_dict["channel_parameters"]["non_lin_coeff"]),
        str(sim_param_dict["channel_parameters"]["iq_imbalance"]),
        str(sim_param_dict["m_c_parameters"]["min_eb_n0"]),
        str(sim_param_dict["m_c_parameters"]["max_eb_n0"]),
        str(sim_param_dict["m_c_parameters"]["step_db"]),
    )

    # Creation of the PHY Layer
    phy_layer = PHY_Layer(emiter, receiver, awgn_channel)

    nb_tries = 0
    ind_eb_n0 = 0

    # Launch the simulation
    while (ind_eb_n0 < len(eb_n0_db)) and (
        not (ind_eb_n0 > 0) or (ber[ind_eb_n0 - 1] > 0 and nb_tries < max_test)
    ):
        # Init variables
        nb_tries = 0
        nb_frame_error = 0
        global_error_nb = 0
        # Set the snr for the channel
        phy_layer.channel.set_var(
            eb_n0_db[ind_eb_n0], sim_param_dict["modulation"]["modulation_order"]
        )
        # For the moment, we consider that the noise variance is not estimated
        # but is Genie aided.
        if sim_param_dict["equalizer"] == "MMSE":
            receiver.equalizer.set_noise_var(phy_layer.channel.var)
        # Monte-Carlo method

        while (nb_tries < max_test) and (
            nb_frame_error < sim_param_dict["m_c_parameters"]["min_error_frame"]
        ):
            # Generation of the frames
            frame = np.random.randint(0, high=2, size=sim_param_dict["frame_length"])
            # Send the frame to the physical layer
            recieved_frame = phy_layer.process_frame(frame)
            # Counting errors
            errors_num = np.sum(recieved_frame != frame)
            # Look at the number of mistake
            if errors_num > 0:
                # Add the number of frame errors
                nb_frame_error = nb_frame_error + 1
            global_error_nb = global_error_nb + errors_num
            # Increase the number of tries
            nb_tries = nb_tries + 1
        # Update error vectors
        ber[ind_eb_n0] = global_error_nb / (nb_tries * sim_param_dict["frame_length"])
        fer[ind_eb_n0] = nb_frame_error / nb_tries
        #     ser[ind_eb_n0] =
        print(
            "At ",
            np.floor(100 * ind_eb_n0 / len(eb_n0_db)),
            " %",
            ", BER = ",
            ber[ind_eb_n0],
            ", FER = ",
            fer[ind_eb_n0],
            " for  Eb/N0 = ",
            eb_n0_db[ind_eb_n0],
            " dB",
            ", SNR = ",
            snr_db[ind_eb_n0],
            "dB",
            " nb_tries = ",
            nb_tries,
        )
        # Increase the snr index
        ind_eb_n0 += 1
        ber_dict = {
            "sim_param": sim_param_dict,
            "results": {"eb_n0_db": eb_n0_db, "snr_db": snr_db, "ber": ber, "fer": fer},
        }
        # Save results in file
        with open(filename, "wb") as handle:
            pickle.dump(ber_dict, handle)

    # Display results figures
    plt.plot(eb_n0_db, ber, "b")
    plt.yscale("log")
    plt.title("BER restults")
    plt.xlabel("Eb/N0 (dB)")
    plt.ylabel("BER")
    plt.grid(True)
    plt.show()

    plt.plot(snr_db, fer, "b")
    plt.yscale("log")
    plt.title("FER restults")
    plt.xlabel("SNR (dB)")
    plt.ylabel("FER")
    plt.grid(True)
    plt.show()

In [13]:
simulation_param_dict = {
    "m_c_parameters": {
        "min_error_frame": 100,
        "targeted_fer": 1e-3,
        "step_db": 2,
        "min_eb_n0": 0,
        "max_eb_n0": 40,
    },
    "channel_parameters": {
        "non_lin_coeff": 0,
        "iq_imbalance": None,
        "channel_taps": np.array([1, 2, 3, 2, 1]),
    },
    "frame_length": 256,
    "modulation": {
        "modulation_order": 4,
        "nb_carriers": 64,
        "cp_length": 8,
        "off_carrier": 0,
    },
    "equalizer": "MMSE",
    "channel_coding": {
        "mem_size": np.array([2]),
        "g_matrix": np.array([[0o5, 0o7]]),
        "rho": 1 / 2,  #  Coding rate
    },
}

monte_carlo_simulation(simulation_param_dict)

At  0.0  % , BER =  0.45906 , FER =  1.0  for  Eb/N0 =  0  dB , SNR =  0.0 dB  nb_tries =  100
At  5.0  % , BER =  0.40772 , FER =  1.0  for  Eb/N0 =  2  dB , SNR =  2.0 dB  nb_tries =  100
At  10.0  % , BER =  0.34262 , FER =  1.0  for  Eb/N0 =  4  dB , SNR =  4.0 dB  nb_tries =  100
At  15.0  % , BER =  0.29519 , FER =  1.0  for  Eb/N0 =  6  dB , SNR =  6.0 dB  nb_tries =  100
At  20.0  % , BER =  0.25512 , FER =  1.0  for  Eb/N0 =  8  dB , SNR =  8.0 dB  nb_tries =  100
At  25.0  % , BER =  0.21533 , FER =  1.0  for  Eb/N0 =  10  dB , SNR =  10.0 dB  nb_tries =  100
At  30.0  % , BER =  0.16585 , FER =  1.0  for  Eb/N0 =  12  dB , SNR =  12.0 dB  nb_tries =  100
At  35.0  % , BER =  0.11207 , FER =  1.0  for  Eb/N0 =  14  dB , SNR =  14.0 dB  nb_tries =  100
At  40.0  % , BER =  0.07641 , FER =  1.0  for  Eb/N0 =  16  dB , SNR =  16.0 dB  nb_tries =  100
At  45.0  % , BER =  0.05248 , FER =  1.0  for  Eb/N0 =  18  dB , SNR =  18.0 dB  nb_tries =  100
At  50.0  % , BER =  0.03602 , F

KeyboardInterrupt: 

In [31]:
def create_data_set(dataset_param_dict):
    """ To create a data set and save it.
    ::
    """
    # Creation of the trellis
    trellis = Trellis(
        dataset_param_dict["channel_coding"]["mem_size"],
        dataset_param_dict["channel_coding"]["g_matrix"],
    )

    # Creation of the emiter
    emiter = Emiter(
        cp_len=dataset_param_dict["modulation"]["cp_length"],
        nb_carriers=dataset_param_dict["modulation"]["nb_carriers"],
        modulation_order=dataset_param_dict["modulation"]["modulation_order"],
        trellis=trellis,
        nb_off_carriers=dataset_param_dict["modulation"]["off_carrier"],
    )

    # Creation of the receiver
    receiver = Receiver(
        cp_len=dataset_param_dict["modulation"]["cp_length"],
        nb_carriers=dataset_param_dict["modulation"]["nb_carriers"],
        modulation_order=dataset_param_dict["modulation"]["modulation_order"],
        trellis=trellis,
        nb_off_carriers=dataset_param_dict["modulation"]["off_carrier"],
        equalizer_type=dataset_param_dict["equalizer"],
    )

    # Creation of the AWGN Channel
    channel = AWGN_Channel(
        mean=0,
        var=0,
        non_lin_coeff=dataset_param_dict["channel_parameters"]["non_lin_coeff"],
        iq_imbalance=dataset_param_dict["channel_parameters"]["iq_imbalance"],
        channel_taps=dataset_param_dict["channel_parameters"]["channel_taps"],
    )

    # Set of the variable
    channel.set_var(
        dataset_param_dict["eb_n0_db"],
        dataset_param_dict["modulation"]["modulation_order"],
    )

    # Generation of the frames
    frame = np.random.randint(0, high=2, size=dataset_param_dict["frame_length"])
    # Encode the frame
    enc_frame = emiter.encode(frame)
    # Modulate the frame
    mod_frame = emiter.modulate_frame(enc_frame)
    # Go through the channel
    channel_frame = channel.get_trough(mod_frame)
    # Demodulation of the received frame
    demod_frame = receiver.demodulate_frame(channel_frame, demod_type="hard")

    # Since we get through the channel, we formate the data for a tensor
    # We reshape the frame at the reception
    nb_ofdm_group = len(mod_frame) // (
        dataset_param_dict["modulation"]["cp_length"]
        + dataset_param_dict["modulation"]["nb_carriers"]
    )
    target_samples = np.reshape(
        mod_frame,
        (
            nb_ofdm_group,
            (
                dataset_param_dict["modulation"]["cp_length"]
                + dataset_param_dict["modulation"]["nb_carriers"]
            ),
        ),
    )

    # Convert complex values into real and cast into a tensor
    target_samples = torch.from_numpy(from_complex_to_real(target_samples))
    print(target_samples)
    samples = torch.from_numpy(from_complex_to_real(demod_frame))
    print(samples)

    # Save results in file
    filename = "./data_set/OFDM_non_lin_coeff_{}_iq_im_{}_eb_n0_{}_proakis_C.pt".format(
        str(dataset_param_dict["channel_parameters"]["non_lin_coeff"]),
        str(dataset_param_dict["channel_parameters"]["iq_imbalance"]),
        str(dataset_param_dict["eb_n0_db"]),
    )

    torch.save(target_samples, filename)
    
    print("Data set created at " + filename)

In [32]:
data_set_generation_param_dict = {
    "eb_n0_db": 10,
    "channel_parameters": {
        "non_lin_coeff": 0,
        "iq_imbalance": None,
        "channel_taps": np.array([1, 2, 3, 2, 1]),
    },
    "frame_length": 1000,
    "modulation": {
        "modulation_order": 4,
        "nb_carriers": 64,
        "cp_length": 8,
        "off_carrier": 0,
    },
    "equalizer": "MMSE",
    "channel_coding": {
        "mem_size": np.array([2]),
        "g_matrix": np.array([[0o5, 0o7]]),
        "rho": 1 / 2,  #  Coding rate
    },
}

create_data_set(data_set_generation_param_dict)

/home/eric/.local/lib/python3.6/site-packages/commpy/modulation.py:77: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  index_list))


tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0404,  0.0214, -0.1156],
       dtype=torch.float64)


AxisError: axis 1 is out of bounds for array of dimension 1

False